In [1]:
import argparse
import torch
import torch.nn as nn
import numpy as np
import os
# from MLP import MLP 
import pickle
# from data_loader import load_test_dataset, data_loader
from torch.autograd import Variable 
import math
import time
import pandas as pd
import pickle

In [2]:
# Load the scaler object from the saved file
with open(r"Scalers_and_weights\scaler.pkl", "rb") as f:
    scaler = pickle.load(f)
    
    
    

In [3]:
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()        
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=2, padding=1), 
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1), 
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64 , 128 , 3 , stride = 2 , padding = 1),
            nn.ReLU(),
            nn.Conv2d(128 , 256 , 3 , stride = 2 , padding = 1),
            nn.ReLU(),
            nn.Conv2d(256 , 512 , 3 , stride = 2 , padding = 1),
            nn.ReLU(),
            nn.Conv2d(512 , 1024 , 2),
            nn.ReLU(),
            nn.Conv2d(1024 , 2048 , 1),
            nn.ReLU(),
            nn.Conv2d(2048 , 28 , 1)
        )
        
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(28, 2048, 1),
            nn.ReLU(),
            nn.ConvTranspose2d(2048, 1024, 1),
            nn.ReLU(),
            nn.ConvTranspose2d(1024, 512, 2),
            nn.ReLU(),
            nn.ConvTranspose2d(512, 256,  3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
            
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [4]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.nn.utils.rnn import pack_padded_sequence
from torch.autograd import Variable

class MLP(nn.Module):
    def __init__(self, input_size, output_size):
        super(MLP, self).__init__()
        self.fc = nn.Sequential(
		nn.Linear(input_size, 1280),nn.PReLU(),nn.Dropout(),
		nn.Linear(1280, 1024),nn.PReLU(),nn.Dropout(),
		nn.Linear(1024, 896),nn.PReLU(),nn.Dropout(),
		nn.Linear(896, 768),nn.PReLU(),nn.Dropout(),
		nn.Linear(768, 512),nn.PReLU(),nn.Dropout(),
		nn.Linear(512, 384),nn.PReLU(),nn.Dropout(),
		nn.Linear(384, 256),nn.PReLU(), nn.Dropout(),
		nn.Linear(256, 256),nn.PReLU(), nn.Dropout(),
		nn.Linear(256, 128),nn.PReLU(), nn.Dropout(),
		nn.Linear(128, 64),nn.PReLU(), nn.Dropout(),
		nn.Linear(64, 32),nn.PReLU(),
		nn.Linear(32, output_size))
        


    def forward(self, x):
        out = self.fc(x)
        return out

In [5]:
def distance(node, point):
        dist = np.sqrt((node[0] - point[0])**2 + (node[1] - point[1])**2)         
        return dist


def goalFound(point , goal):
        if distance(goal, point) <= 5:
            return True
        return False
    
def to_var(x, volatile=False):
    if torch.cuda.is_available():
        x = x.to(torch.device("cuda"))
    return x


In [8]:
import cv2
# from data_loader import Autoencoder
# from MLP import MLP
from torchvision import datasets, transforms
import torch
import numpy

# Encoding the image
autoencoder = Autoencoder()
Q = autoencoder.encoder
Q.load_state_dict(torch.load(r"Scalers_and_weights\model_982.pkl"))
if torch.cuda.is_available():
    Q = Q.to(torch.device("cuda"))
    
# Load trained model for path generation
mlp = MLP(32, 2)
mlp.load_state_dict(torch.load(r"Scalers_and_weights\model_190.pkl"))

if torch.cuda.is_available():
    mlp.cuda()

# Load test dataset
image_path = r"Datasets\images\152.jpg"
path = []

image = cv2.imread(image_path ,  cv2.IMREAD_GRAYSCALE)

encoded_w_m=np.zeros((1,28),dtype=np.float32)
to_tensor = transforms.ToTensor()
torch_img = to_tensor(image)


if torch.cuda.is_available():
    torch_img = torch_img.to(torch.device("cuda")) 

output=Q(torch_img)
output = output.squeeze()
output=output.data.cpu()
output = output.numpy()
encoded_w_m[0] = output

start_x = 20
start_y = 100
start = np.array([[start_x, start_y]])
start = scaler.transform(start)
goal_x = 40
goal_y = 30
goal = np.array([[goal_x, goal_y]])
goal = scaler.transform(goal)
# start = np.array([start_x, start_y])
# goal = np.array([goal_x, goal_y])
path.append(torch.tensor(np.array([start[0][0], start[0][1]])))
# path.append(start)


start1=torch.from_numpy(start)
goal1=torch.from_numpy(goal)
obs = torch.from_numpy(encoded_w_m)

start1 = start1.float()
goal1 = goal1.float()

start1 = start1.squeeze()
goal1 = goal1.squeeze()
obs = obs.squeeze()

# print(start1.shape)
# print(goal1.shape)
# print(obs.shape)
# data = torch.cat((obs,start1,goal1))
# print(data.shape)
# Implement the Network to generate the path
while True:
    data = torch.cat((obs,start1,goal1))
    data = to_var(data)
    current = mlp(data)
    current = current.data.cpu()
    #print(current.numpy())
    path.append(current)
    start1 = current
#     print(current)
    current = current.numpy()
    current = np.array([[current[0], current[1]]])
    current = scaler.inverse_transform(current)
#     print("current",current)
    if goalFound(numpy.array([current[0][0], current[0][1]]) , numpy.array([goal_x, goal_y])):
        path.append(goal1)
        break
path.append(goal1)

In [9]:
path_modified = []
for ele in (path):
    if not isinstance(ele, np.ndarray):
        ele = ele.numpy()
        ele = ele.tolist()
    else :
        ele = ele.tolist()
    path_modified.append(ele)

# print(path_modified)

path_modified = np.array(path_modified)
path_modified = scaler.inverse_transform(path_modified)
print(path_modified)

[[ 20.         100.        ]
 [ 19.34988355  99.79672563]
 [ 16.95722483  98.09593491]
 [ 20.8218307   97.25864094]
 [ 26.94833295  97.60227407]
 [ 26.43146411  96.673574  ]
 [ 37.59685913  94.96665537]
 [ 43.57956371  93.4933058 ]
 [ 47.05037578  89.65841218]
 [ 47.79335117  82.48584434]
 [ 50.85609258  80.91173846]
 [ 49.52615589  75.53183103]
 [ 48.84810388  75.81761956]
 [ 49.946431    75.17229374]
 [ 50.00566201  79.04891396]
 [ 51.04377942  81.44101921]
 [ 57.3314137   79.15921962]
 [ 50.1201957   75.49099408]
 [ 48.44241723  69.51276324]
 [ 45.4878716   67.8646719 ]
 [ 47.09754775  67.42378145]
 [ 50.30920157  65.72060339]
 [ 48.37696895  65.96680209]
 [ 45.29868197  62.42100009]
 [ 43.83022135  59.7523119 ]
 [ 46.56983265  57.53732142]
 [ 45.19504807  53.35539914]
 [ 47.79998548  55.19209921]
 [ 49.32571414  55.57984995]
 [ 50.32150189  51.38799777]
 [ 50.18069861  46.60640967]
 [ 45.30167476  41.41547581]
 [ 44.27487978  43.49963314]
 [ 44.94946683  41.0546163 ]
 [ 42.1927169 